# Data loading

In [24]:
def load_data(data_path):
  data = pd.read_csv(data_path)  
  return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
    print(len(pallet))
trainentest = load_data("demand_kWtrain_val.csv")




1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
4925535


# Python packages




In [2]:
import math, datetime, time, random
import numpy as np
import pandas as pd
from datetime import timedelta
import re
import math, datetime, time, random
import matplotlib.pyplot as plt



# Preprocessing Dataframes
Preprocessing inbound, outbound, pallet and weather 

setting up the features dataframe, giving it a date, time, year, month etc.

Create definition for adding time columns and making a datetime object of a column

In [4]:
def addtimecol(df, colname): ####input df and colname 
        df[colname] = pd.to_datetime(df[colname])         
        df['year'] = df[colname].dt.year
        df['month'] = df[colname].dt.month
        df['weekday'] = df[colname].dt.weekday
        df['day'] = df[colname].dt.day
        df['hour'] = df[colname].dt.hour
        df['minute'] = df[colname].dt.minute        
        return df
def make_datetime(df, col_name): ###To make datetime object from df at colname    
    for index, row in df.iterrows():
        t = df.at[index, col_name]
        time = datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
        localtime = time - (timedelta(hours=6))
        df.at[index, 'localstrptime'] = localtime

Setting up a featurestrain set and add timecolumns, then make that into the csv file

In [25]:
featurestrain = pd.DataFrame({'datetime_local': trainentest['datetime_local'], 'demand_kW' : trainentest['demand_kW']})
addtimecol(featurestrain, 'datetime_local')
#featurestrain = featurestrain.set_index('datetime_local')

#featurestrain.to_csv('featurestrain.csv')

,datetime_local,demand_kW,year,month,weekday,day,hour,minute
0,2018-12-31 21:15:00,2064.101392,2018,12,0,31,21,15
1,2018-12-31 21:30:00,1874.002081,2018,12,0,31,21,30
2,2018-12-31 21:45:00,1988.168511,2018,12,0,31,21,45
3,2018-12-31 22:00:00,2022.795943,2018,12,0,31,22,0
4,2018-12-31 22:15:00,1986.981872,2018,12,0,31,22,15
...,...,...,...,...,...,...,...,...
365344,2021-12-13 17:55:00,NaN,2021,12,0,13,17,55
365345,2021-12-13 17:56:00,NaN,2021,12,0,13,17,56
365346,2021-12-13 17:57:00,NaN,2021,12,0,13,17,57
365347,2021-12-13 17:58:00,NaN,2021,12,0,13,17,58


In [26]:
allfeatures_df = featurestrain.copy()

In [5]:
base_ft = load_data('featuretrain.csv')
base_ft = addtimecol(base_ft, 'datetime_local')
base_ft

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00


Preprocessing inbound dataframe 

## Here I chose to drop the 4000 rows for which load_start_datetime didn't have a value
I would prefer to fill in the truck signin datetime then

In [12]:
inbound = load_data("inbound_loads.csv")
inbound_sec = inbound.dropna(subset = ['net_weight', 'load_start_datetime'])
#inbound_sec = inbound_sec.dropna(axis = 0, how = 'any', thresh = 1, subset = ['load_start_datetime', 'truck_signin_datetime'], inplace = True)
inbound_sec = inbound_sec.drop('Unnamed: 0', axis = 1)
inbound_sec = inbound_sec.sort_values('load_start_datetime')
inbound_sec['load_start_datetime'] = pd.to_datetime(base_inbound['load_start_datetime'])
inbound_sec = inbound_sec.set_index('load_start_datetime')

inbound_sec

NameError: name 'base_inbound' is not defined

In [13]:
#droplevels where carriercode is cancel or None because then you cannot estimate the temp. //its not that many
#droplevels where load_start and truck_signin are both missing because useless info then

inbound_indexes = inbound[inbound['carrier_code'] == "CANCEL"].index
inbound.drop(inbound_indexes, inplace = True)
inbound.dropna(subset = ['customer_code'], inplace = True)
inbound.dropna(axis = 0, how = 'any', thresh = 1, subset = ['load_start_datetime', 'truck_signin_datetime'], inplace = True)

Preprocessing weather dataframe

In [34]:
base_weather = load_data('weather.csv') 

#Add localstrptime, 6 hours before UTC
#base_weather = make_datetime(base_weather, 'datetime_UTC')
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])

#Remove unnecessary columns/change column names
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1', 'hour', 'datetime', 'datetime_UTC'], axis=1)




In [40]:
base_weather = base_weather.set_index('datetime_local')
allfeatures_df = allfeatures_df.set_index('datetime_local')


,demand_kW,year,month,weekday,day,hour,minute
datetime_local,,,,,,,
2018-12-31 21:15:00,2064.101392,2018,12,0,31,21,15
2018-12-31 21:30:00,1874.002081,2018,12,0,31,21,30
2018-12-31 21:45:00,1988.168511,2018,12,0,31,21,45
2018-12-31 22:00:00,2022.795943,2018,12,0,31,22,0
2018-12-31 22:15:00,1986.981872,2018,12,0,31,22,15
...,...,...,...,...,...,...,...
2021-12-13 17:55:00,NaN,2021,12,0,13,17,55
2021-12-13 17:56:00,NaN,2021,12,0,13,17,56
2021-12-13 17:57:00,NaN,2021,12,0,13,17,57


In [41]:
#Concatenate the weather DataFrame to the base DataFrame
trainentest = pd.concat([allfeatures_df, base_weather], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

,datetime_local,demand_kW,year,month,weekday,day,hour,minute,datetime_local,Relative Humidity,Temperature
0,2018-12-31 21:15:00,2064.101392,2018,12,0,31,21,15,2018-12-31 18:00:00,50.37,53.6
1,2018-12-31 21:30:00,1874.002081,2018,12,0,31,21,30,2018-12-31 18:05:00,50.37,53.6
2,2018-12-31 21:45:00,1988.168511,2018,12,0,31,21,45,2018-12-31 18:10:00,50.37,53.6
3,2018-12-31 22:00:00,2022.795943,2018,12,0,31,22,0,2018-12-31 18:15:00,50.37,53.6
4,2018-12-31 22:15:00,1986.981872,2018,12,0,31,22,15,2018-12-31 18:20:00,50.37,53.6
...,...,...,...,...,...,...,...,...,...,...,...
365344,2021-12-13 17:55:00,NaN,2021,12,0,13,17,55,NaT,NaN,NaN
365345,2021-12-13 17:56:00,NaN,2021,12,0,13,17,56,NaT,NaN,NaN
365346,2021-12-13 17:57:00,NaN,2021,12,0,13,17,57,NaT,NaN,NaN
365347,2021-12-13 17:58:00,NaN,2021,12,0,13,17,58,NaT,NaN,NaN


In [16]:
#base_ft = base_weather


pandas._libs.tslibs.timestamps.Timestamp

Creating features Dataframe

In [19]:
featurestrain = load_data('featurestrain.csv')
featurestrain['datetime_local'] = pd.to_datetime(featurestrain['datetime_local']) 
featurestrain = featurestrain.set_index('datetime_local')

Combine weather data with featurestrain: Temperature and Humidity

In [11]:
df1 = base_ft.sort_values('datetime_local') 
df2 = base_weather.sort_values('datetime_local')
fttrain = pd.concat([df1, df2], axis = 1)

Drop irrelevant levels waar NA in staat

In [14]:
fttrain.dropna(subset={'year', 'Temperature'})      

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,...,datetime,Relative Humidity,Temperature,datetime_UTC,year,month,weekday,day,hour,minute
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,...,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00,2018.0,12.0,0.0,31.0,18.0,0.0
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,...,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00,2018.0,12.0,0.0,31.0,18.0,5.0
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,...,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00,2018.0,12.0,0.0,31.0,18.0,10.0
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,...,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00,2018.0,12.0,0.0,31.0,18.0,15.0
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,...,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00,2018.0,12.0,0.0,31.0,18.0,20.0
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,...,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00,2018.0,12.0,0.0,31.0,18.0,25.0
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,...,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00,2018.0,12.0,0.0,31.0,18.0,30.0
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,...,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00,2018.0,12.0,0.0,31.0,18.0,35.0
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,...,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00,2018.0,12.0,0.0,31.0,18.0,40.0
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,...,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00,2018.0,12.0,0.0,31.0,18.0,45.0


# First Feature - Warmth movement
quantify warmth movement in terms of pallet movement in pallet DataFrame

First, testing what pallet movement categories are relevant
Considerations so far:
what is interesting data for where warm orders go?
        Move_category: outbound_move = not interesting because orders have already been stored(time between receive date and t is big)
        inbound_move = ; location_move = 
        received_date: skip too old packages because they are in the right temp. to store --> too old = longer than 72h. in?
        to_location_code: CPsth = per definitie een outbound move. DRsth = per definitie een outbound move
        work_type; MOVE = location_move; PUTAWAYs = inbound, close to entrance; 

# Plannen - start - inbound moves
for move category/work_type, make a DataFrame where palletcount is given per location 'category'

How many pallets are moved towards a location?
1. create testpal dataframe, make sure transaction_date/receive_date is a datetime object, remove rows where received date is beyond 72 hours because then it is out of the blast freezer maximally. (maybe 72 hours is a tad too strict, a week may be better)
2. for testing: create testpal_sec and add the three letters 
2. add to testpal create tomovcat and frommovcat 3 letters per move category (I dont get this to work yet)
3. create new dataframe and store how many pallets moved towards a tomovcat
4. create for each unique movecat a row in a dataframe and count the amount of pallets going there
5. concat the base_ft and palmov df


# 1.
Create testpal dataframe

In [9]:
testpal = pd.DataFrame({'from_location_code' : pallet['from_location_code'], 'move_category' : pallet['move_category'], 'received_date' : pallet['received_date'], 'customer_code' : pallet['customer_code'],  'to_location_code' : pallet['to_location_code'],  'work_type' : pallet['work_type'],  'transaction_date' : pallet['transaction_date'], 'quantity' : pallet["quantity"]    })
testpal = testpal[testpal['move_category'] == 'inbound move']
testpal['datetime_local'] = pd.to_datetime(testpal['transaction_date'])
testpal['received_datetime'] = pd.to_datetime(testpal['received_date'])
testpal = testpal.drop(testpal[((testpal['received_datetime'] + timedelta(days = 5)) < testpal['datetime_local'])].index)

# 2 
create testpal section to test with


In [16]:
testpal_sec = testpal[0:1000]

In [17]:
testpal_sec

,from_location_code,move_category,received_date,customer_code,to_location_code,work_type,transaction_date,quantity,datetime_local,received_datetime
164,DOOR 59,inbound move,2021-11-01 13:36:13,2537,0110230774,PUTAWAYS,2021-11-01 18:42:05,90,2021-11-01 18:42:05,2021-11-01 13:36:13
179,DOOR 59,inbound move,2021-11-01 13:37:55,2537,0110230792,PUTAWAYS,2021-11-01 18:50:43,90,2021-11-01 18:50:43,2021-11-01 13:37:55
184,DOOR 59,inbound move,2021-11-01 13:40:44,2537,0110220194,PUTAWAYS,2021-11-01 18:48:14,93,2021-11-01 18:48:14,2021-11-01 13:40:44
189,DOOR 59,inbound move,2021-11-01 13:41:07,2537,0110230782,PUTAWAYS,2021-11-01 18:38:21,112,2021-11-01 18:38:21,2021-11-01 13:41:07
196,DOOR 59,inbound move,2021-11-01 13:41:28,2537,0110211454,PUTAWAYS,2021-11-01 18:33:59,112,2021-11-01 18:33:59,2021-11-01 13:41:28
198,DOOR 59,inbound move,2021-11-01 13:41:46,2537,0110210521,PUTAWAYS,2021-11-01 18:31:19,112,2021-11-01 18:31:19,2021-11-01 13:41:46
202,DOOR 59,inbound move,2021-11-01 13:42:11,2537,0110231453,PUTAWAYS,2021-11-01 18:29:09,112,2021-11-01 18:29:09,2021-11-01 13:42:11
204,DOOR 59,inbound move,2021-11-01 13:42:33,2537,0110230942,PUTAWAYS,2021-11-01 18:26:51,112,2021-11-01 18:26:51,2021-11-01 13:42:33
206,DOOR 59,inbound move,2021-11-01 13:43:25,2537,0110230592,PUTAWAYS,2021-11-01 18:24:32,112,2021-11-01 18:24:32,2021-11-01 13:43:25
213,DOOR 59,inbound move,2021-11-01 13:50:05,2537,0110230184,PUTAWAYS,2021-11-01 17:54:26,112,2021-11-01 17:54:26,2021-11-01 13:50:05


# 2.
Make column for category frommovcat and tomovcat in testpal and testpal_sec

In [18]:
##just for now to make it quick(this one works)
for index, row in testpal_sec.iterrows():  
        a = testpal_sec.at[index, 'to_location_code']
        b = testpal_sec.at[index, 'from_location_code']
        testpal_sec.loc[index, 'frommovcat'] = b[0:3]
        testpal_sec.loc[index, 'tomovcat'] = a[0:3]

C:\Users\sonja\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\sonja\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
def addtofrom(palletdf):
    newpalletdf = palletdf.copy()
    for index, row in palletdf.iterrows():  
        a = palletdf.at[index, 'to_location_code']        
        newpalletdf.loc[index, 'tomovcat'] = a[0:3]
        b = palletdf.at[index, 'from_location_code']
        palletdf.loc[index, 'frommovcat'] = b[0:3]
    return newpalletdf


Create dataframe section for each worktype

In [20]:
inunl = testpal[testpal['work_type'] == 'UN-LOAD'] 
inputa = testpal[testpal['work_type'] == 'PUTAWAYS'] 
ininb = testpal[testpal['work_type'] == 'INBOUNDRUN'] 

Create frommovcat and tomovcat for each dataframe

In [325]:
##This doesn't work
ininb = addtofrom(ininb)
inputa = addtofrom(inputa)
inunl = addtofrom(inunl)

# 3
Getting it to work for specific case

In [16]:
movedpal = pd.DataFrame({'datetime_local': testpal_sec['datetime_local']})
for index, row in new_ft.iterrows():
    time = new_ft.at[index, 'datetime_local']    
    mask = (movedpal['datetime_local'] > time) & (testpal_sec['datetime_local'] <= time + timedelta(hours = 24))
#    movedpal.at[index, "003"] = testpal_sec[testpal_sec['tomovcat'] == "003"].quantity.loc[mask].sum()
    #relevant.at[index, 'total_weight'] = base_inbound.net_weight.loc[mask].sum()


NameError: name 'testpal_sec' is not defined

# 4 
write a generic function where you get the pallets moved with a certain worktype towards all its unique locations.
You want 
1. you make a dataframe with the length of the featuresdataframe
2. get the section of palletdf for which a column has a certain value
3. for every datetime in the ftdf, get the sum of pallets brought into each unique location category
4. concat the created dataframe and the base_ft dataframe

In [276]:
def makepalmov(ftdf, palletdf, column, nameofvalue, delta):
    palmov = pd.DataFrame({'datetime_local': ftdf['datetime_local']})    
    section = palletdf[palletdf[column] == nameofvalue]
    print(section)
    for x in section.tomovcat.unique():
        for index, row in ftdf.iterrows():            
            time = ftdf.at[index, 'datetime_local']            
            mask = (palmov['datetime_local'] > time) & (palmov['datetime_local'] <= time + delta)
            palmov.at[index, f"tomov{x}"] = palletdf.quantity.loc[mask].sum() 
    return palmov
             
    

In [324]:
brug = timedelta(hours = 24)
palletmov = makepalmov(new_ft, testpal_sec, 'work_type', "UN-LOAD", brug)

In [49]:
palletmov

NameError: name 'palletmov' is not defined

Do it for all unique work_types of palletdf inbound moves

brug = timedelta(hours = 24) ##The timebridge you wanna use

In [322]:
palmovput = makepalmov(new_ft, testpal_sec, 'work_type', "PUTAWAYS", brug)
palmovin = makepalmov(new_ft, testpal_sec, 'work_type', "INBOUNDRUN", brug)
palmovun = makepalmov(new_ft, testpal_sec, 'work_type', "UN-LOAD", brug)

# 5. - concat new feature to feature df

In [327]:
new_ft = pd.concat([new_ft, palmovput], axis = 1)
new_ft = pd.concat([new_ft, palmovin], axis = 1)
new_ft = pd.concat([new_ft, palmovun], axis = 1)

Here I was testing stuff, you may ignore it xx

In [106]:
outb = testpal[testpal['move_category'] == 'outbound move']
inb = testpal[testpal['move_category'] == 'inbound move']
loc = testpal[testpal['move_category'] == 'location move']
movout = outb['work_type'].unique()
movin = inb['work_type'].value_counts()
movloc = loc['work_type'].unique()
print('moveout:     ', movout)
print('movin:     ', movin)
print('movloc:     ', movloc)

moveout:      ['WRONGFULL' 'DROPS' 'PARTIALS-V' 'FULLS' 'FULLCHGLOC' 'STAGE'
 'CHRYPICK-S' 'ANYFULL' 'PARTIALS-S' 'SWAPFULL' 'DROPRELO' 'SWAPDROP'
 'FULLRELO' 'REVDROPS' 'DROPRETURN']
movin:      PUTAWAYS      1064119
INBOUNDRUN      61493
UN-LOAD          8353
Name: work_type, dtype: int64
movloc:      ['MOVE' 'LOCCHANGE' 'RELOS' 'RELORUN']


Within inbound move/location move, what movements are there and which are relevant? 

Findings: 

Inbound: 

UN-LOAD, orders mainly coming from going to DR0.. and going to DR0... 

PUTAWAYS, orders mainly coming from door numbers, and going to 0... number

INBOUNDRUN, orders mainly coming from DR0.., and going to large variety of places, inverter, dr.., 

In [111]:
#selecting part of dataset for which work_type == ''
inunl = testpal[testpal['work_type'] == 'UN-LOAD'] 
inputa = testpal[testpal['work_type'] == 'PUTAWAYS'] 
ininb = testpal[testpal['work_type'] == 'INBOUNDRUN'] 
#print('inunl:     ', inunl['to_location_code'].value_counts())
#print('inputa:     ', inputa['to_location_code'].value_counts())
#print('ininb:     ', ininb['to_location_code'].value_counts())

In [240]:
###Merge featuretrain and weather dataset based on datetime_local
def make_datetime(df, col_name): ###To make datetime object from df at colname
    for index, row in df.iterrows():
        t = df.at[index, col_name]
        time = datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
        localtime = time - (timedelta(hours=6))
        df.at[index, col_name] = localtime
     
df1 = featurestrain.sort_values('datetime_local')
df2 = weather.sort_values('datetime_local')
fttrain = pd.concat([df1, df2], axis = 1)
fttrain.to_csv('featuretrain.csv')

NameError: name 'fttrain' is not defined

In [134]:
print(type(weather['datetime_local'].iloc[0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


# Second Feature - Warmth coming in the Warehouse
At each time, how much warm product is coming into the warehouse?

1. create new_ft dataframe to add the added warmth to, make it into datetime objects
2. create base_inbound dataframe to mess with in this section
3. create def. addwarmth that creates a dataframe for each time entry in new_ft and maps the entered warmth between that time and delta
4. do it directly first
5. concat relevant and new_ft

# 1

In [10]:
###1
new_ft = base_ft.copy()
new_ft['datetime_local'] = pd.to_datetime(new_ft['datetime_local'])
new_ft

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00


# 2

In [11]:
base_inbound = inbound.copy()
base_inbound['startpoint'] = pd.to_datetime(base_inbound['load_start_datetime'])

In [41]:
inbound_sec

,warehouse_order_number,customer_code,load_reference_number,carrier_code,front_temperature,middle_temperature,back_temperature,net_weight,weight_uom,case_quantity,pallet_count,load_finish_datetime,signout_datetime,dock_door_number,trailer_number,truck_signin_datetime
load_start_datetime,,,,,,,,,,,,,,,,
2018-12-30 06:33:00,1315527.0,899.0,I42168974,CHEF,-5.0,-6.0,-6.0,42796.0,pounds,1950.0,24.0,2018-12-30 09:38:00,2018-12-30 05:38:15,1.0,U69689,2018-12-30 05:38:14
2018-12-31 11:45:00,1315640.0,7240.0,I42169019,TYSF,0.0,0.0,0.0,900.0,pounds,60.0,2.0,2018-12-31 11:51:00,2018-12-31 09:48:50,44.0,29261,2018-12-31 09:48:49
2018-12-31 14:01:00,1312923.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,8640.0,pounds,288.0,4.0,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312927.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,1170.0,pounds,39.0,1.0,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312928.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,2940.0,pounds,98.0,1.0,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312932.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,6690.0,pounds,223.0,3.0,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312934.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,5850.0,pounds,195.0,2.0,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312935.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,270.0,pounds,9.0,1.0,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44
2018-12-31 14:01:00,1312940.0,8187.0,I42168556,CARG,-2.0,0.0,-3.0,15596.0,pounds,520.0,7.0,2018-12-31 14:25:00,2018-12-31 14:30:20,47.0,21736,2018-12-31 08:33:44


# 3


In [28]:
def addwarmth(ftdf, delta): ###make a dataframe for each row in ftdf and add that to the relevant dataframe so they can be concatted
    relevant = pd.DataFrame({'datetime_local': ftdf['datetime_local']})
    for index, row in ftdf.iterrows():
        time = ftdf.at[index, 'datetime_local']
        mask = (base_inbound['startpoint'] > time) & (base_inbound['startpoint'] <= time + timedelta(delta))
        relevant.at[index, 'total_weight'] = base_inbound.net_weight.loc[mask].sum() 

# 4
Run thisone when having a large break!

In [ ]:
relevant = pd.DataFrame({'datetime_local': new_ft['datetime_local']})
for index, row in new_ft.iterrows():
    time = new_ft.at[index, 'datetime_local']
    mask = (base_inbound['startpoint'] < time) & (base_inbound['startpoint'] >= time - timedelta(hours = 24))
    relevant.at[index, 'total_weight'] = base_inbound.net_weight.loc[mask].sum()
print(relevant)

# This takes too long, try this code:

In [257]:
inbound_sec

,Unnamed: 0,warehouse_order_number,customer_code,load_reference_number,carrier_code,front_temperature,middle_temperature,back_temperature,net_weight,weight_uom,case_quantity,pallet_count,load_start_datetime,load_finish_datetime,signout_datetime,dock_door_number,trailer_number,truck_signin_datetime
startpoint,,,,,,,,,,,,,,,,,,
2019-02-11 04:42:00,0,1290649.0,2537.0,I42171197,TMDN,-1.0,-1.0,-1.0,39000.0,pounds,1625.0,24.0,2019-02-11 04:42:00,2019-02-11 04:51:00,2019-02-11 04:19:10,59.0,1362664,2019-02-11 02:00:09
2019-01-16 10:11:00,1,1293130.0,2537.0,I42169605,CKKI,4.0,4.0,4.0,29561.0,pounds,1232.0,26.0,2019-01-16 10:11:00,2019-01-16 11:27:00,2019-01-16 09:57:04,60.0,MWCU5258117,2019-01-16 07:24:26
2019-07-30 09:59:00,2,1298947.0,2537.0,I42178319,CKTH,NaN,NaN,NaN,39984.0,pounds,1666.0,24.0,2019-07-30 09:59:00,2019-07-30 10:18:00,2019-07-30 10:21:11,61.0,CGMU5027645,2019-07-30 07:21:09
2019-01-10 08:51:00,3,1299947.0,2537.0,I42169266,NTGO,-10.0,-7.0,-6.0,37800.0,pounds,1260.0,18.0,2019-01-10 08:51:00,2019-01-10 08:58:00,NaN,58.0,54087,2019-01-10 08:04:05
2019-01-10 07:26:00,4,1299948.0,2537.0,I42169269,NTGO,3.0,4.0,5.0,41394.0,pounds,1380.0,18.0,2019-01-10 07:26:00,2019-01-10 08:08:00,2019-01-10 07:43:34,56.0,305266,2019-01-10 04:29:02
2019-01-10 08:36:00,5,1299949.0,2537.0,I42169268,NTGO,1.0,1.0,1.0,39000.0,pounds,1300.0,18.0,2019-01-10 08:36:00,2019-01-10 09:20:00,2019-01-10 08:19:26,52.0,1011,2019-01-10 05:34:34
2019-01-10 06:14:00,6,1299950.0,2537.0,I42169270,NTGO,-8.0,-8.0,-7.0,40196.0,pounds,1340.0,20.0,2019-01-10 06:14:00,2019-01-10 06:39:00,2019-01-10 08:44:13,55.0,53143,2019-01-10 03:38:59
2019-10-24 09:13:00,7,1301040.0,8873.0,I42182788,CPU,NaN,NaN,NaN,1680.0,pounds,56.0,3.0,2019-10-24 09:13:00,2019-10-24 09:17:00,2019-10-24 09:32:25,35.0,16418,2019-10-24 07:18:29
2019-01-09 13:09:00,16,1305206.0,8873.0,I42168834,BKMP,NaN,NaN,NaN,21924.0,pounds,756.0,19.0,2019-01-09 13:09:00,2019-01-09 13:22:00,2019-01-09 13:32:08,42.0,8,2019-01-09 11:05:01


In [256]:
inbound_sec = base_inbound.copy()
inbound_sec = inbound_sec.set_index('startpoint')

In [270]:

inbound_resampled_data_samen = inbound_sec.net_weight.resample('1440Min', base = 0, closed = 'right').sum()
print(inbound_resampled_data_samen)
for minute in range(0,360):
    df2 = inbound_sec.net_weight.resample('1440Min', base = minute, closed = 'right').sum()    
    inbound_resampled_data_samen = pd.concat([inbound_resampled_data_samen, df2])

startpoint
2018-12-30      42796.0
2018-12-31     295346.0
2019-01-01      63040.0
2019-01-02    1049929.0
2019-01-03    1486933.0
2019-01-04    1457688.0
2019-01-05     807761.0
2019-01-06     440142.0
2019-01-07    1016374.0
2019-01-08    1505010.0
2019-01-09    1528615.0
2019-01-10    1459986.0
2019-01-11    1501973.0
2019-01-12     699416.0
2019-01-13     528311.0
2019-01-14    2068894.0
2019-01-15    1751494.0
2019-01-16    1793455.0
2019-01-17    2081048.0
2019-01-18    2143427.0
2019-01-19    1282844.0
2019-01-20     441003.0
2019-01-21    2325394.0
2019-01-22    2023809.0
2019-01-23    1905970.0
2019-01-24    1983788.0
2019-01-25    3483578.0
2019-01-26     940766.0
2019-01-27     638057.0
2019-01-28    1881207.0
                ...    
2021-12-05     693392.0
2021-12-06    1636557.0
2021-12-07    1599827.0
2021-12-08    1313036.0
2021-12-09    1588037.0
2021-12-10    1935009.0
2021-12-11     998521.0
2021-12-12     761717.0
2021-12-13    1856989.0
2021-12-14    1252659.0
2021-

In [282]:
inbound_resampled_data_samen.head(12375)

startpoint
2018-12-30 00:00:00      42796.0
2018-12-30 00:00:00      42796.0
2018-12-30 00:01:00      42796.0
2018-12-30 00:02:00      42796.0
2018-12-30 00:03:00      42796.0
2018-12-30 00:04:00      42796.0
2018-12-30 00:05:00      42796.0
2018-12-30 00:06:00      42796.0
2018-12-30 00:07:00      42796.0
2018-12-30 00:08:00      42796.0
2018-12-30 00:09:00      42796.0
2018-12-30 00:10:00      42796.0
2018-12-30 00:11:00      42796.0
2018-12-30 00:12:00      42796.0
2018-12-30 00:13:00      42796.0
2018-12-30 00:14:00      42796.0
2018-12-30 00:15:00      42796.0
2018-12-30 00:16:00      42796.0
2018-12-30 00:17:00      42796.0
2018-12-30 00:18:00      42796.0
2018-12-30 00:19:00      42796.0
2018-12-30 00:20:00      42796.0
2018-12-30 00:21:00      42796.0
2018-12-30 00:22:00      42796.0
2018-12-30 00:23:00      42796.0
2018-12-30 00:24:00      42796.0
2018-12-30 00:25:00      42796.0
2018-12-30 00:26:00      42796.0
2018-12-30 00:27:00      42796.0
2018-12-30 00:28:00      42796.0

In [292]:
#inbound_sec.sort_values(by = index)

KeyError: 13479

In [276]:
base_ft

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00


In [272]:
inbound_resampled_data_samen = inbound_resampled_data_samen.sort_index()

# 5
Concat with dataframe/make standalone feature df with all times

In [27]:
#relevant = load_data('warmthft.csv')
#relevant = relevant.drop(['Unnamed: 0'], axis = 1)
relevant.to_csv('warmthft.csv')

In [25]:
#warmthft = pd.concat([base_ft, relevant], axis = 1)

# 6 Check for correctness 
YES! you can now skip this code :)


In [54]:
relevant_sec = pd.DataFrame({'datetime_local': new_ft['datetime_local']})
new_ft_sec = new_ft[10020:10021]
for index, row in new_ft_sec.iterrows():
    time = new_ft_sec.at[index, 'datetime_local']
    print(time)
    mask = (base_inbound['startpoint'] < time) & (base_inbound['startpoint'] >= time - timedelta(hours = 24))
   
    lijst = mask[mask == True].index
    totaal = 0
    for x in lijst:
        totaal += base_inbound.net_weight.loc[x]
        print(base_inbound.net_weight.loc[x])
    print(totaal)
    som = base_inbound.net_weight.loc[mask].sum()
    print(som)
    relevant_sec.at[index, 'total_weight'] = som 
print(relevant_sec[relevant_sec['total_weight'] > 1])

2019-02-01 12:35:00
39993.0
35580.0
36840.0
40950.0
8250.0
38257.0
37890.0
22985.0
1080.0
972.0
28300.0
29826.0
38609.0
29866.0
40560.0
36324.0
36720.0
7128.0
27055.0
34551.0
38103.0
29302.0
29584.0
27000.0
27000.0
31325.0
32365.0
4920.0
4920.0
31798.0
25032.0
25620.0
36167.0
8237.0
46012.0
37800.0
37800.0
4920.0
25564.0
36274.0
17275.0
46080.0
33480.0
22631.0
46080.0
45376.0
37800.0
36057.0
27000.0
32891.0
29076.0
40583.0
12650.0
31360.0
47628.0
39653.0
45404.0
40778.0
16685.0
34492.0
40177.0
43700.0
1878335.0
1878335.0
           datetime_local  total_weight
10020 2019-02-01 12:35:00     1878335.0


In [42]:
#warmthfeature.head(100)


time = new_ft.at[1340, 'datetime_local']
mask = (base_inbound['startpoint'] < time) & (base_inbound['startpoint'] >= time - timedelta(hours = 24))
warmth = base_inbound.net_weight.loc[mask].sum()
print(mask[mask == True])
print(warmth)

#2019-01-04 21:45:00,820921.0

17     True
393    True
404    True
412    True
416    True
445    True
450    True
457    True
458    True
462    True
465    True
471    True
474    True
476    True
478    True
479    True
481    True
482    True
483    True
485    True
486    True
487    True
488    True
489    True
491    True
497    True
498    True
499    True
502    True
Name: startpoint, dtype: bool
820921.0


In [45]:
print(inbound.loc[17])

Unnamed: 0                                 17
warehouse_order_number            1.30521e+06
customer_code                            8873
load_reference_number               I42168835
carrier_code                             BKMP
front_temperature                           1
middle_temperature                          2
back_temperature                            2
net_weight                              31668
weight_uom                             pounds
case_quantity                            1092
pallet_count                               26
load_start_datetime       2019-01-05 07:05:00
load_finish_datetime      2019-01-05 07:19:00
signout_datetime          2019-01-05 07:33:55
dock_door_number                           42
trailer_number                           T-30
truck_signin_datetime     2019-01-05 06:06:48
Name: 17, dtype: object


In [6]:
####for information in pallet
#base_pallet = pallet[pallet['move_category'] == 'inbound move']
#base_pallet = pallet.copy()
base_pallet = base_pallet.drop(['source_system_id', 'warehouse_facility_id'], axis = 1)



NameError: name 'base_pallet' is not defined

# Feature 3: pallet movements (within 5 minutes) of time
1. create pallet dataframe and filter through irrelevant information

2. Write general function

3. data exploration. Wordt location move altijd gedaan naar een fork of vanaf een fork? 

# Create dataframes
1. Create pallet_sec, a section of the columns of pallet_df
2. feature_palletmove is the base_ft df plus the row added to it

In [55]:
pallet_sec = pallet.copy()
#pallet_sec = pallet_sec[pallet_sec['quantity'] != 0]
pallet_sec = pallet_sec.drop(['Unnamed: 0','tran_type', 'Unnamed: 0.1', 'source_system_id', 'warehouse_facility_id', 'final_pallet_code', 'lot_code'], axis = 1)

#pallet_sec['transaction_date'] = pd.to_datetime(pallet_sec['transaction_date'])
pallet_sec = pallet_sec.reset_index()

#remove previous not-working index
pallet_sec = pallet_sec.drop(['index'], axis = 1)

In [64]:
feature_palletmove = base_ft.copy()

Makes sure that datetime_local and transaction_date are datetime objects

In [66]:
feature_palletmove['datetime_local'] = pd.to_datetime(feature_palletmove['datetime_local'])
feature_palletmove['datetime_local'] = pd.Series(feature_palletmove.datetime_local.dt.to_pydatetime(), dtype='O')
pallet_sec['transaction_date'] = pd.to_datetime(pallet_sec['transaction_date'])
pallet_sec['transaction_date']= pd.Series(pallet_sec.transaction_date.dt.to_pydatetime(), dtype='O')


# 2 Make General Function
feature_palletmove datetime is time dus dat is de tijd die je ingeeft
je wilt totaan die tijd en 5 minuten ervoor

gevraagd movement delta
selectie = time - delta  toten met  time
movement moet dus kleiner zijn dan time - delta 
mask tijden na time en tijden voor time plus delta

In [89]:
feature_palletmove

,datetime_local,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC
0,2018-12-31 18:00:00,2018,12,0,31,18,0,2018-12-31 18:00:00-06:00,50.37,53.60,2019-01-01 00:00:00
1,2018-12-31 18:05:00,2018,12,0,31,18,5,2018-12-31 18:05:00-06:00,50.37,53.60,2019-01-01 00:05:00
2,2018-12-31 18:10:00,2018,12,0,31,18,10,2018-12-31 18:10:00-06:00,50.37,53.60,2019-01-01 00:10:00
3,2018-12-31 18:15:00,2018,12,0,31,18,15,2018-12-31 18:15:00-06:00,50.37,53.60,2019-01-01 00:15:00
4,2018-12-31 18:20:00,2018,12,0,31,18,20,2018-12-31 18:20:00-06:00,50.37,53.60,2019-01-01 00:20:00
5,2018-12-31 18:25:00,2018,12,0,31,18,25,2018-12-31 18:25:00-06:00,50.37,53.60,2019-01-01 00:25:00
6,2018-12-31 18:30:00,2018,12,0,31,18,30,2018-12-31 18:30:00-06:00,50.37,53.60,2019-01-01 00:30:00
7,2018-12-31 18:35:00,2018,12,0,31,18,35,2018-12-31 18:35:00-06:00,53.81,51.80,2019-01-01 00:35:00
8,2018-12-31 18:40:00,2018,12,0,31,18,40,2018-12-31 18:40:00-06:00,53.81,51.80,2019-01-01 00:40:00
9,2018-12-31 18:45:00,2018,12,0,31,18,45,2018-12-31 18:45:00-06:00,50.37,53.60,2019-01-01 00:45:00


Call function pallet_movements

In [71]:
def pallet_movements(featuredf, palletdf, delta):
    palmov = pd.DataFrame({'datetime_local': featuredf['datetime_local']})

    for index, row in featuredf.iterrows():        
        time = featuredf.at[index, 'datetime_local']            
        mask = (palletdf['transaction_date'] <= time) & (palletdf['transaction_date'] >= time - delta) ##the mask contains the indexes you want
        som = palletdf.quantity.loc[mask].sum()
        pallet_movement_df.at[index, 'pallet_movements'] = som ##you calc the sum of the quantities that are within the mask        
    return palmov

In [94]:
brug = timedelta(minutes = 5)
#pallet_movements(feature_palletmove[0:1000], pallet_sec[0:1000], brug)


,datetime_local,pallet_movements
0,2018-12-31 18:00:00,0.0
1,2018-12-31 18:05:00,0.0
2,2018-12-31 18:10:00,0.0
3,2018-12-31 18:15:00,0.0
4,2018-12-31 18:20:00,0.0
5,2018-12-31 18:25:00,0.0
6,2018-12-31 18:30:00,0.0
7,2018-12-31 18:35:00,0.0
8,2018-12-31 18:40:00,0.0
9,2018-12-31 18:45:00,0.0


# This code above takes too long
1. order palletmove df into chronological order so that if time in featuredf is bigger than, escape.
2. pallet_sec drop quantity is 0 entries
3. reset index of pallet_sec

In [97]:
pallet_sec = pallet_sec[pallet_sec['quantity'] != 0]
pallet_sec = pallet_sec.sort_values(by = 'transaction_date')

In [102]:
pallet_sec = pallet_sec.reset_index() ##reset index and drop unnecessary indexes
pallet_sec = pallet_sec.drop(['index', 'level_0'], axis = 1)
pallet_sec

,customer_code,from_location_code,to_location_code,transaction_date,quantity,work_type,warehouse_pallet_code,received_date,move_category
0,899,DOOR 47,0090020791,2019-01-02 00:29:12,63,PUTAWAYS,1315707001-0010,2019-01-02 00:05:14,inbound move
1,899,DOOR 47,0090020791,2019-01-02 00:31:18,63,PUTAWAYS,1315707001-0009,2019-01-02 00:05:05,inbound move
2,899,DOOR 47,0090031115,2019-01-02 00:33:01,63,PUTAWAYS,1315707001-0008,2019-01-02 00:04:54,inbound move
3,899,DOOR 47,0090031174,2019-01-02 00:34:56,63,PUTAWAYS,1315707001-0007,2019-01-02 00:04:45,inbound move
4,899,DOOR 47,0090031202,2019-01-02 00:36:28,63,PUTAWAYS,1315707001-0006,2019-01-02 00:04:36,inbound move
5,899,DOOR 47,0090031245,2019-01-02 00:37:54,63,PUTAWAYS,1315707001-0005,2019-01-02 00:04:24,inbound move
6,899,DOOR 47,0090040602,2019-01-02 00:39:48,63,PUTAWAYS,1315707001-0011,2019-01-02 00:05:34,inbound move
7,899,DOOR 57,0090011071,2019-01-02 00:40:09,60,PUTAWAYS,1315706001-0009,2019-01-01 23:57:13,inbound move
8,899,DOOR 47,0090040602,2019-01-02 00:41:55,63,PUTAWAYS,1315707001-0012,2019-01-02 00:05:44,inbound move
9,899,DOOR 57,0090011071,2019-01-02 00:42:49,60,PUTAWAYS,1315706001-0010,2019-01-01 23:57:29,inbound move


Rewrite the pallet_movements function ideas:
1. let it go over pallet_sec and fill in quantity in palmov dataframe. 
2. write second function that calls

In [172]:
time = pallet_movement_df.at[100106, 'datetime_local'] 
brug = timedelta(minutes = 5)
#print(pallet_sec[pallet_sec.transaction_date > time - timedelta(minutes = 5)].head(20))

#for row in pallet_sec.itertuples():     
#    if row.transaction_date >= time-brug: 
#        return row

#criteria = pallet_sec.transaction_date>=time-brug
print(pallet_sec.iloc[((pallet_sec.transaction_date >= time-brug).idxmax()) : (((pallet_sec.transaction_date >= time-brug).idxmax()&(pallet_sec.transaction_date < time).idxmax())) ])


Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
2019-11-14 11:15:00


In [190]:
pallet_sec_sec = pallet_sec.copy()
pallet_sec_sec = pallet_sec_sec.set_index('transaction_date')

In [231]:
pallet_sec_sec


####voor 2019-01-02 min 48

,customer_code,from_location_code,to_location_code,quantity,work_type,warehouse_pallet_code,received_date,move_category
transaction_date,,,,,,,,
2019-01-02 00:29:12,899,DOOR 47,0090020791,63,PUTAWAYS,1315707001-0010,2019-01-02 00:05:14,inbound move
2019-01-02 00:31:18,899,DOOR 47,0090020791,63,PUTAWAYS,1315707001-0009,2019-01-02 00:05:05,inbound move
2019-01-02 00:33:01,899,DOOR 47,0090031115,63,PUTAWAYS,1315707001-0008,2019-01-02 00:04:54,inbound move
2019-01-02 00:34:56,899,DOOR 47,0090031174,63,PUTAWAYS,1315707001-0007,2019-01-02 00:04:45,inbound move
2019-01-02 00:36:28,899,DOOR 47,0090031202,63,PUTAWAYS,1315707001-0006,2019-01-02 00:04:36,inbound move
2019-01-02 00:37:54,899,DOOR 47,0090031245,63,PUTAWAYS,1315707001-0005,2019-01-02 00:04:24,inbound move
2019-01-02 00:39:48,899,DOOR 47,0090040602,63,PUTAWAYS,1315707001-0011,2019-01-02 00:05:34,inbound move
2019-01-02 00:40:09,899,DOOR 57,0090011071,60,PUTAWAYS,1315706001-0009,2019-01-01 23:57:13,inbound move
2019-01-02 00:41:55,899,DOOR 47,0090040602,63,PUTAWAYS,1315707001-0012,2019-01-02 00:05:44,inbound move


In [250]:
#grouped = pallet_sec_sec.groupby(lambda x: (x.year, x.month, x.day, x.hour))
minute_resampled_data_samen = pallet_sec_sec.quantity.resample('5Min', base = 0, closed = 'right').sum()
print(minute_resampled_data_samen)
for minute in range(1,5):    
    df2 = pallet_sec_sec.quantity.resample('5Min', base = minute, closed = 'right').sum()    
    minute_resampled_data_samen = pd.concat([minute_resampled_data_samen, df2])


transaction_date
2019-01-02 00:25:00      63
2019-01-02 00:30:00     189
2019-01-02 00:35:00     189
2019-01-02 00:40:00     246
2019-01-02 00:45:00     492
2019-01-02 00:50:00     366
2019-01-02 00:55:00     309
2019-01-02 01:00:00     369
2019-01-02 01:05:00     309
2019-01-02 01:10:00     337
2019-01-02 01:15:00     630
2019-01-02 01:20:00    1080
2019-01-02 01:25:00    1200
2019-01-02 01:30:00     770
2019-01-02 01:35:00     600
2019-01-02 01:40:00     765
2019-01-02 01:45:00     453
2019-01-02 01:50:00     960
2019-01-02 01:55:00    1200
2019-01-02 02:00:00    1200
2019-01-02 02:05:00       0
2019-01-02 02:10:00       0
2019-01-02 02:15:00       0
2019-01-02 02:20:00       0
2019-01-02 02:25:00       0
2019-01-02 02:30:00       0
2019-01-02 02:35:00       0
2019-01-02 02:40:00       0
2019-01-02 02:45:00       0
2019-01-02 02:50:00       0
                       ... 
2022-01-11 13:25:00     142
2022-01-11 13:30:00       0
2022-01-11 13:35:00       0
2022-01-11 13:40:00       0
202

In [239]:
minute_resampled_data_samen.index
minute_resampled_data_samen = pd.to_datetime(minute_resampled_data_samen.index)

AttributeError: 'DatetimeIndex' object has no attribute 'index'

In [249]:
minute_resampled_data_samen.sort_index()

transaction_date
2019-01-02 00:25:00     63
2019-01-02 00:26:00     63
2019-01-02 00:27:00    126
2019-01-02 00:28:00    126
2019-01-02 00:29:00    189
2019-01-02 00:30:00    189
2019-01-02 00:31:00    189
2019-01-02 00:32:00    189
2019-01-02 00:33:00    252
2019-01-02 00:34:00    189
2019-01-02 00:35:00    189
2019-01-02 00:36:00    249
2019-01-02 00:37:00    249
2019-01-02 00:38:00    246
2019-01-02 00:39:00    309
2019-01-02 00:40:00    246
2019-01-02 00:41:00    309
2019-01-02 00:42:00    369
2019-01-02 00:43:00    369
2019-01-02 00:44:00    369
2019-01-02 00:45:00    492
2019-01-02 00:46:00    429
2019-01-02 00:47:00    429
2019-01-02 00:48:00    369
2019-01-02 00:49:00    429
2019-01-02 00:50:00    366
2019-01-02 00:51:00    369
2019-01-02 00:52:00    369
2019-01-02 00:53:00    369
2019-01-02 00:54:00    369
                      ... 
2022-01-11 15:21:00    120
2022-01-11 15:22:00    120
2022-01-11 15:23:00    120
2022-01-11 15:24:00     60
2022-01-11 15:25:00    120
2022-01-11 

In [238]:
minute_resampled_data_samen

DatetimeIndex(['2019-01-02 00:25:00', '2019-01-02 00:30:00',
               '2019-01-02 00:35:00', '2019-01-02 00:40:00',
               '2019-01-02 00:45:00', '2019-01-02 00:50:00',
               '2019-01-02 00:55:00', '2019-01-02 01:00:00',
               '2019-01-02 01:05:00', '2019-01-02 01:10:00',
               ...
               '2022-01-11 15:04:00', '2022-01-11 15:09:00',
               '2022-01-11 15:14:00', '2022-01-11 15:19:00',
               '2022-01-11 15:24:00', '2022-01-11 15:29:00',
               '2022-01-11 15:34:00', '2022-01-11 15:39:00',
               '2022-01-11 15:44:00', '2022-01-11 15:49:00'],
              dtype='datetime64[ns]', name='transaction_date', length=1592126, freq=None)

In [202]:
pd.Period('2018', freq = '1Min')

Period('2018-01-01 00:00', 'T')

In [204]:
pallet_sec_sec.resample('1M', how={'pallets_moved' : pallet_sec_sec.quantity.sum})


C:\Users\sonja\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  """Entry point for launching an IPython kernel.


TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [127]:
def pallet_movements(featuredf, palletdf, delta):
    palmov = pd.DataFrame({'datetime_local': featuredf['datetime_local']})    
    for index, row in featuredf.iterrows():         
        for row in palletdf.itertuples():
        # Define your criteria here
            if row.transaction_date >= time-delta 
            return row
        
            time = featuredf.at[index, 'datetime_local']   
            palletdf_sec = palletdf[(pallet_sec.transaction_date >= time - delta) & (pallet_sec.transaction_date < time)].all

            print(palletdf_sec)
        #mask = palletdf_sec['transaction_date'] <= time ##the mask contains the indexes you want
        
            som = palletdf.quantity.loc[mask].sum()
            pallet_movement_df.at[index, 'pallet_movements'] = som ##you calc the sum of the quantities that are within the mask        
    return palmov

In [128]:
brug = timedelta(minutes = 5)
pallet_movements(pallet_movement_df, pallet_sec, brug)

Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date

Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date

Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date

Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date, quantity, work_type, warehouse_pallet_code, received_date, move_category]
Index: []
Empty DataFrame
Columns: [customer_code, from_location_code, to_location_code, transaction_date

KeyboardInterrupt: 

In [ ]:
    for index, row in featuredf.iterrows():        
        time = featuredf.at[index, 'datetime_local']   
        
        mask = (palletdf['transaction_date'] <= time) & (palletdf['transaction_date'] >= time - delta) ##the mask contains the indexes you want
        som = palletdf.quantity.loc[mask].sum()
        pallet_movement_df.at[index, 'pallet_movements'] = som ##you calc the sum of the quantities that are within the mask        
    return palmov

In [91]:
pallet_movement_df = pd.DataFrame({'datetime_local': feature_palletmove['datetime_local']})
pallet_movement_df

,datetime_local
0,2018-12-31 18:00:00
1,2018-12-31 18:05:00
2,2018-12-31 18:10:00
3,2018-12-31 18:15:00
4,2018-12-31 18:20:00
5,2018-12-31 18:25:00
6,2018-12-31 18:30:00
7,2018-12-31 18:35:00
8,2018-12-31 18:40:00
9,2018-12-31 18:45:00


# 3 Correctness check

Using a small subset of feature_palletmove df (only 3 values)

Checking if feature_palletmove is correct, unnecessary code

In [80]:
brug = timedelta(minutes = 5)
feature_palletmove_sec = feature_palletmove[13469:13480] ##subset of featurepalletmove is created

In [81]:
print(pallet_movements(feature_palletmove_sec, pallet_sec, brug))

            datetime_local  pallet_movements
13469  2019-02-12 05:53:00            1295.0
13470  2019-02-12 05:55:00            1579.0
13471  2019-02-12 06:00:00            1481.0
13472  2019-02-12 06:05:00            2159.0
13473  2019-02-12 06:10:00            1769.0
13474  2019-02-12 06:15:00            1189.0
13475  2019-02-12 06:20:00            1024.0
13476  2019-02-12 06:25:00             200.0
13477  2019-02-12 06:30:00             758.0
13478  2019-02-12 06:35:00             553.0
13479  2019-02-12 06:40:00             184.0


# YES! it works :)

In [84]:
#scores = pd.DataFrame({'datetime_local': featuredf['datetime_local']})

for index, row in feature_palletmove_sec.iterrows():        
    time = feature_palletmove_sec.at[index, 'datetime_local']  
    print(time)
    mask = (pallet_sec['transaction_date'] <= time) & (pallet_sec['transaction_date'] >= time - brug) ##the mask contains the indexes you want
    lijst = mask[mask == True].index
    print(lijst)
    totaal = 0
    for x in lijst:
        print(pallet_sec.transaction_date[x])
        print(pallet_sec.quantity.loc[x])
        totaal += pallet_sec.quantity.loc[x]
    print('som:     ', pallet_sec.quantity.loc[mask].sum())
    

2019-02-12 05:53:00
Int64Index([3439373, 3525692, 3623724, 3623726, 3623740, 3623750, 3623760,
            3623770, 3623776, 3625843, 3625845, 3625847, 3625932, 3627102,
            3627112, 3627115],
           dtype='int64')
2019-02-12 05:48:10
19
2019-02-12 05:51:28
60
2019-02-12 05:48:41
120
2019-02-12 05:48:19
91
2019-02-12 05:50:53
120
2019-02-12 05:52:50
120
2019-02-12 05:51:11
120
2019-02-12 05:48:19
24
2019-02-12 05:48:19
5
2019-02-12 05:50:33
119
2019-02-12 05:50:03
119
2019-02-12 05:50:03
119
2019-02-12 05:50:34
112
2019-02-12 05:48:13
49
2019-02-12 05:51:41
49
2019-02-12 05:51:53
49
som:      1295
2019-02-12 05:55:00
Int64Index([3525692, 3623734, 3623740, 3623750, 3623760, 3623772, 3625843,
            3625845, 3625847, 3625923, 3625925, 3625932, 3625947, 3625953,
            3627112, 3627115],
           dtype='int64')
2019-02-12 05:51:28
60
2019-02-12 05:54:42
120
2019-02-12 05:50:53
120
2019-02-12 05:52:50
120
2019-02-12 05:51:11
120
2019-02-12 05:53:15
120
2019-02-12 05

In [49]:
pallet_movement_df

Make it into a feature_df

In [68]:
pallet_movement_df.to_csv('pallet_moved_feature')

# WARNING!!!!

# Below is data exploration code. I didnt want to remove but I know Rob is a bit sensitive to unorganized code, so this is a warning for him

In [14]:
forks_from = pd.Series(from_location_list).str.match(r'.*FORK.*')
#forks_in = pd.Series(to_location_list).str.match(r'.*FORK.*')
df.drop(df.filter(regex='Test').columns, axis=1)

NameError: name 'from_location_list' is not defined

In [100]:
#forks = pallet_sec.drop(pallet_sec.filter(regex = '.*FORK.*'), axis = 1)

C:\Users\sonja\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  


In [101]:
forks

,customer_code,from_location_code,to_location_code,transaction_date,quantity,work_type,warehouse_pallet_code,received_date,move_category
443515,899,DOOR 47,0090020791,2019-01-02 00:29:12,63,PUTAWAYS,1315707001-0010,2019-01-02 00:05:14,inbound move
443513,899,DOOR 47,0090020791,2019-01-02 00:31:18,63,PUTAWAYS,1315707001-0009,2019-01-02 00:05:05,inbound move
443511,899,DOOR 47,0090031115,2019-01-02 00:33:01,63,PUTAWAYS,1315707001-0008,2019-01-02 00:04:54,inbound move
443509,899,DOOR 47,0090031174,2019-01-02 00:34:56,63,PUTAWAYS,1315707001-0007,2019-01-02 00:04:45,inbound move
443507,899,DOOR 47,0090031202,2019-01-02 00:36:28,63,PUTAWAYS,1315707001-0006,2019-01-02 00:04:36,inbound move
443504,899,DOOR 47,0090031245,2019-01-02 00:37:54,63,PUTAWAYS,1315707001-0005,2019-01-02 00:04:24,inbound move
443517,899,DOOR 47,0090040602,2019-01-02 00:39:48,63,PUTAWAYS,1315707001-0011,2019-01-02 00:05:34,inbound move
443453,899,DOOR 57,0090011071,2019-01-02 00:40:09,60,PUTAWAYS,1315706001-0009,2019-01-01 23:57:13,inbound move
443519,899,DOOR 47,0090040602,2019-01-02 00:41:55,63,PUTAWAYS,1315707001-0012,2019-01-02 00:05:44,inbound move
443455,899,DOOR 57,0090011071,2019-01-02 00:42:49,60,PUTAWAYS,1315706001-0010,2019-01-01 23:57:29,inbound move


In [14]:




####estimating the current warmth of incoming products

###from the inbound datafame, add column 'estimate_temperature' to add the median temp. given the customer code

###make a dictionary with customer code and its median temperature
######get the median middle temperature of each customer code into a dataframe so you can 
warmthdf = pd.DataFrame({'customer_code': inbound['customer_code'], 'carrier_code': inbound['carrier_code'], 'middle_temperature': inbound['middle_temperature']})
    ###filter through missing values/droplevel at missing values
warmthdf = warmthdf.dropna()

warmthdict = {}
for customer in warmthdf.customer_code.unique():      ###per customercode, give the median middle_temperature
    median = warmthdf.loc[warmthdf['customer_code'] == customer]
    warmthdict[f'{customer}']=  median['middle_temperature'].median()


In [15]:
warmthdict['new'] = warmthdf['middle_temperature'].median() ###add median temp for the customer_codes that are new

###iterate over inbound and fill in the estimate_temperature 
for index, row in inbound.iterrows():
    if inbound.at[index, 'middle_temperature'] >-5000: ####fill in the middle_temperature if that is known
        inbound.at[index, 'estimate_temp'] = inbound.at[index, 'middle_temperature']
       
    else: ###else fill in the estimated temp. that belongs to the customer_code, unless it is not in the dict., then fill in 0.0 which is the median temp. of all the orders
        code = inbound.at[index, 'customer_code']        
        median = warmthdict.get(f'{code}', 0.0) 
        inbound.at[index, 'estimate_temp'] = median

In [16]:
warmthdict

{'2537.0': 0.0,
 '8873.0': 0.0,
 '14104.0': -1.5,
 '625.0': 0.0,
 '8187.0': -1.0,
 '5810.0': -1.0,
 '6134.0': -2.0,
 '6910.0': -1.0,
 '899.0': 1.0,
 '7240.0': -1.0,
 '874.0': 0.0,
 '114276.0': 0.0,
 '7475.0': 33.0,
 '1296.0': -3.0,
 '17296.0': -2.0,
 '7289.0': 32.5,
 '2319.0': 2.0,
 '117006.0': 0.0,
 '10377.0': -8.0,
 '1980.0': -1.0,
 '10235.0': -1.5,
 '1059.0': 0.0,
 '3610.0': -1.0,
 '1987.0': -1.0,
 '11383.0': -6.0,
 '615.0': 4.0,
 '8713.0': 0.0,
 '6147.0': -2.0,
 'new': 0.0}

In [133]:
###To check the different variances per customer_code: boxplot the different customer_code temperatures
warmthdf.assign(index=df.groupby('customer_code').cumcount()).pivot('index','customer_code','middle_temperature').plot(kind='box', figsize = [30,30])


In [77]:
###pallet history contains where packages are going. Which packages are interesting? Testing above. Making it into a feature:

###How many packages are entering a certain area at t as a feature for each location-type, we first make location types to and from
    ###
    ###How much time must be considered? 24-72h for blastfreezer, ideal: between in and out.
    ###What locations to consider? Ignore forks/dock/
    ###what locations can be considered the same, ideal: what locations have same temp.
    
    ##preprocessing the data:
    ###some packages are already in the warehouse for a long time
        #don't consider movements if the time between entranc and t is more than 72 hours because won't cost extra energy 
    



#df = df.assign(movcat = lambda x: (x[0:3]))
#df = df.assign(movcat = lambda x: (x['to_location_code'][0:3])

df = pallet[10000:50000]
for index, row in df.iterrows():  
        a = df.at[index, 'to_location_code']
        b = df.at[index, 'from_location_code']
        df.loc[index, 'frommovcat'] = b[0:3]
        df.loc[index, 'tomovcat'] = a[0:3]
df['tomovcat'].value_counts()


###per movement type I will make one feature that defines how many pallets are moved towards a location. 
    #df = df.assign(Product=lambda x: (x['Field_1'] * x['Field_2'] * x['Field_3']))
    

###divide moving history into different category of areas. 
#mapping = {'(/^FORK/gm)': 'Fork', '(/^CFORK/gm)': 'CFORK', '(/^30/gm)' : '30', '(/^CP/gm)' : 'CP',  '(/^50/gm)' : '50',  '(/^15/gm)' : '15', '(/^11/gm)' : '11',  '(/^01/gm)' : '01' 
           

    ### amount+1 for each order that entered the building within 24h
#            if t <= inbound.at[index, 'load_start_datetime'] <= newtime :
#                amount += measurewarmth(index, row)                            ### use measurewarmth to estimate the warmth that an entry brings in
                
#    return amount


003    8608
FOR    4982
004    4347
015    2640
005    2607
CFO    2459
002    2216
DR0    2051
012    1911
009    1370
010    1199
CP9    1088
CP2    1037
011     913
INV     712
001     599
CP1     399
CP6     241
WRA     221
014     151
013      58
DR9      49
CP7      45
SPJ      41
263      26
000      11
CP4       8
DR5       7
UNA       4
Name: tomovcat, dtype: int64

In [ ]:

###pallet types of locations:
#003    2130
#FOR    1177 
#012    1116
#011    1116
#004    1024
#005     619
#CP9     566
#DR0     483
#CP2     371
#CFO     282
#010     200
#009     174
#INV     142
#SPJ     133
#CP1     126
#CP6     109
#002      69
#WRA      60
#001      47
#015      40
#CP7       8
#CP4       5
#DR9       2


# Model training and testing

